In [5]:
import torch
import torch.nn as nn
import torchvision.models as models

In [282]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        ## encoder
        self.encoder = models.squeezenet1_1(pretrained=True)
        self.encoder.classifier = nn.Sequential(nn.Conv2d(512, 1, kernel_size=(2, 2), stride=(1, 1)))
        
        ## decoder
        self.decoder = nn.Sequential( nn.ConvTranspose2d(1, 16, 4, stride=2),
                                       nn.BatchNorm2d(16),
                                       nn.ReLU(),
                                       nn.ConvTranspose2d(16, 32, 3, stride=2),
                                       nn.BatchNorm2d(32),
                                       nn.ReLU(),
                                       nn.ConvTranspose2d(32, 64, 3, stride=2),
                                       nn.BatchNorm2d(64),
                                       nn.ReLU(),
                                       nn.ConvTranspose2d(64, 32, 3, stride=2),
                                       nn.BatchNorm2d(32),
                                       nn.ReLU(),
                                       nn.ConvTranspose2d(32, 16, 3, stride=1),
                                       nn.BatchNorm2d(16),
                                       nn.ReLU(),
                                       nn.ConvTranspose2d(16, 1, 8, stride=1),)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x.view(1,1,12,12))
        return x
        

In [283]:
model = Autoencoder()